In [5]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/train/', 
    classes=['other','samsung'],
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary', 
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/test/',
    classes=['other', 'samsung'],
    target_size=(150,150),
    batch_size=100,
    class_mode='binary', 
)

Found 18752 images belonging to 2 classes.
Found 8331 images belonging to 2 classes.


In [6]:
from keras import models 
from keras import layers 

# VGG16 합성곱 기반 층 만들기 

from keras.applications import vgg16
from keras import optimizers

# 합성곱 기반 층
conv_base = vgg16.VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

# 모델 정의
model = models.Sequential() 
model.add(conv_base)# 합성곱 기반 층
model.add(layers.Flatten()) # 합성곱 기반 층 출력 1차원 텐서(벡터로)변형
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# 합성곱 기반 층 가중치 동결 
conv_base.trainable = False
print(f'{len(model.trainable_weights)}') 


# 모델 컴파일 
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.adam_v2.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=83)

8


2022-02-25 01:10:22.181791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - ETA: 0s - loss: 0.5603 - accuracy: 0.7417

2022-02-25 01:13:36.571585: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - 264s 1s/step - loss: 0.5603 - accuracy: 0.7417 - val_loss: 0.1946 - val_accuracy: 0.9972


In [11]:
from keras.preprocessing import image 
from keras.applications.vgg16 import preprocess_input

img_path = '/Users/kibeomkim/Desktop/samsung6.jpg'

img = image.load_img(img_path, target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

result = model.predict(x) ; print(result)

[[0.99999976]]


In [23]:
import os 
valid_list = os.listdir('/Users/kibeomkim/Desktop/openlogo_split/test/samsung') 

def galaxy(name) : 
    img = image.load_img(f'/Users/kibeomkim/Desktop/openlogo_split/test/samsung/{name}', target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x) 

result = [galaxy(image) for image in valid_list]

In [35]:
print(len(result))
print(len([x for x in result if x > 0.5]))

19
14


In [36]:
result

[array([[1.]], dtype=float32),
 array([[0.00021203]], dtype=float32),
 array([[0.99999976]], dtype=float32),
 array([[0.08828855]], dtype=float32),
 array([[6.498255e-05]], dtype=float32),
 array([[0.00729018]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.86372066]], dtype=float32),
 array([[0.9999995]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.11309852]], dtype=float32),
 array([[0.99999654]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.98967326]], dtype=float32),
 array([[0.8654114]], dtype=float32),
 array([[1.]], dtype=float32)]